# Toward Optimal Retrieval: Dynamically Choosing `k` in Vector-Based Search


## 📚 Introduction to Retrieval-Augmented Generation (RAG)

### What is RAG?

**Retrieval-Augmented Generation (RAG)** is a hybrid architecture that combines the strengths of **retrieval-based** systems and **generative language models**. Instead of relying solely on a language model's internal parameters to answer questions or generate content, RAG explicitly augments the generation process by retrieving relevant external knowledge.

The RAG architecture was introduced to address limitations in traditional language models, especially their tendency to **"hallucinate"** or produce incorrect information due to lack of grounding in external knowledge sources.

---

### 🔍 Why Use RAG?

Pre-trained language models (e.g., GPT, LLaMA, etc.) are trained on massive datasets but are inherently static:

- They cannot learn new knowledge after training unless fine-tuned.
- Their knowledge is limited to their training cutoff.
- They struggle with domain-specific or long-tail queries.

RAG solves these problems by:
- **Fetching real-time or up-to-date information** from external sources like documents, databases, or knowledge graphs.
- **Reducing hallucinations** by grounding responses in retrieved context.
- **Improving performance** on specialized tasks or domains without retraining the model.

---

### 🧠 How Does RAG Work?

The RAG pipeline generally consists of **three main components**:

1. **Encoder-based Retriever**
   - Given an input query, a retriever fetches the top-*k* most relevant documents from a vectorDB (e.g., FAISS, ChromaDB) using vector similarity search.
   - These documents serve as an external knowledge source.

2. **Contextual Fusion**
   - The retrieved documents are passed alongside the query into a language model (e.g., a transformer decoder) to generate an informed response.
   - This can be done by concatenating the documents and query into a single prompt.

3. **Generator**
   - A generative model (like GPT or BART) produces the final output based on the augmented input (query + retrieved knowledge).

```text
User Query → Retriever → Top-k Documents → Generator → Final Answer


# ❗ Problem - Limitations of Static `k` in Vector Similarity Search

## Background

In Retrieval-Augmented Generation (RAG) and other retrieval-based systems, **vector similarity search** is a key operation. Given a query, the system retrieves the top-*k* most similar documents from a vector database using semantic embeddings and similarity metrics (e.g., cosine similarity or dot product).

The parameter **`k`** represents the number of documents to retrieve per query. In most RAG implementations, `k` is set to a **fixed value** (e.g., `k=3` or `k=5`) for all inputs.

While simple and easy to implement, **using a static value of `k` across all inputs introduces several problems** that can negatively impact retrieval relevance, model performance, and computational efficiency.

---

## 🔍 The Core Problem: One-Size-Does-Not-Fit-All

Different user queries or prompts have different levels of complexity, ambiguity, and knowledge requirements. However, a static `k` assumes that **every query benefits equally from the same number of retrieved documents** — which is often not true.

---

## 🔥 Why a Fixed `k` is Suboptimal

### 1. **Under-retrieval (k too small)**
- Important context may be **missed**, especially for complex or vague queries.
- Language model generates **incomplete or hallucinated** responses due to lack of sufficient information.
- Example: A legal or medical question might require 10+ documents to cover relevant information.

### 2. **Over-retrieval (k too large)**
- Irrelevant or noisy documents may dilute the useful context.
- More documents → longer input prompt → higher **token costs** in LLMs.
- May confuse the model, especially when irrelevant docs are included.
- Wastes compute, memory, and latency for simple, narrow queries.

### 3. **No Adaptivity to Query Entropy or Difficulty**
- Not all queries are created equal:
  - Some are **simple and factoid-like** ("What is the capital of Italy?")
  - Others are **ambiguous, multi-faceted, or domain-specific**
- Fixed `k` ignores this variability, leading to suboptimal results in either direction.

---

## 🎯 Research Objective

The goal of this research is to **dynamically determine the optimal value of `k` per query**, based on characteristics of the input or the retrieval results — such as:

- Query entropy or uncertainty
- Query length and type
- Similarity distribution of top retrieved documents
- Historical performance metrics

By intelligently adjusting `k`, we aim to improve:

- Retrieval relevance and precision
- LLM answer quality
- Efficiency and cost-effectiveness of the system

In the next sections, we will explore strategies, algorithms, and evaluation methods for achieving this dynamic retrieval objective.



# 🧪 Approaches to Dynamically Determine `k` in Vector Similarity Search

Choosing the right number of documents (`k`) to retrieve for a given query is crucial for balancing relevance, efficiency, and downstream model performance in RAG systems.

Here we explore three statistically-motivated techniques for determining `k` dynamically based on the **distribution of similarity scores** between the query and candidate documents.

---

## 1. Benjamini–Hochberg Procedure (False Discovery Rate Control)

### 🔍 Overview
The **Benjamini–Hochberg (BH) procedure** is a statistical method for **controlling the False Discovery Rate (FDR)** — the expected proportion of false positives among the selected items. It is typically used in multiple hypothesis testing scenarios.

In the context of vector similarity search:
- Each document can be treated as a "hypothesis" (i.e., "Is this document relevant?").
- We compute **p-values** (or a proxy derived from similarity scores) for each document.
- BH controls the expected rate of false positives among the selected top-*k* documents.

### ⚙️ How it works
1. Convert similarity scores into pseudo p-values (e.g., via ranking or null distribution assumptions).
2. Sort these p-values in ascending order: $( p_1, p_2, ..., p_n )$
3. For each p-value, check:
$
p_i \leq \frac{i}{n} \cdot \alpha
$


   where $\alpha \$ is the desired FDR (e.g., 0.05).
4. Select the **largest `i`** that satisfies the inequality — set `k = i`.

### ✅ Pros
- More power than conservative tests like Bonferroni.
- Controls false discovery rather than per-test error.
- Adapts naturally to the number and quality of candidate documents.

---

## 2. Bonferroni Correction (Family-Wise Error Rate Control)

### 🔍 Overview
The **Bonferroni correction** is a conservative method to control the **Family-Wise Error Rate (FWER)** — the probability of making **any** false discovery.

It is stricter than BH and is often used when **false positives must be avoided at all costs**.

### ⚙️ How it works
1. Convert similarity scores into pseudo p-values.
2. Adjust the threshold using:
   $
   \alpha' = \frac{\alpha}{n}
   $
   where $ n $ is the total number of documents and $\alpha$ is the desired overall error rate (e.g., 0.05).
3. Select all documents with $p_i \leq \alpha'$
4. Set `k` as the number of documents that meet the criterion.

### ⚠️ Caveats
- Very conservative: often results in **low `k`** or even `k=0`, especially when many candidates are noisy or weakly similar.
- Better suited for high-stakes applications where **false positives are expensive**.

---

## 3. Higher Criticism Thresholding

### 🔍 Overview
**Higher Criticism (HC)** is a powerful method for detecting **sparse and weak signals** in large-scale testing problems. It is especially effective when:
- Only a small fraction of documents are truly relevant.
- Their similarity scores are only **slightly stronger than noise**.

Originally proposed by Donoho & Jin (2008), HC finds an **optimal threshold** by balancing signal detection and noise suppression.

📄 [PNAS Article](https://www.pnas.org/doi/abs/10.1073/pnas.0807471105)

### ⚙️ How it works
1. Convert similarity scores into z-scores or p-values under a null model.
2. Compute the **Higher Criticism statistic** for each ordered p-value:
   $
   HC(i) = \frac{\sqrt{n} \left( \frac{i}{n} - p_i \right)}{\sqrt{p_i (1 - p_i)}}
   $
3. Find the **index `i` with the maximum HC value** → this index indicates the optimal threshold.
4. Set `k = i`, selecting the top-`k` documents as relevant.

### 🚀 Advantages
- Adaptive to sparse signal settings.
- Theoretically powerful under weak signal conditions.
- Can outperform both BH and Bonferroni when only a few strong matches exist.

---

## 📊 Summary of Methods

| Method                    | Controls           | Conservative? | Adaptive? | Best Use Case                                  |
|--------------------------|--------------------|---------------|-----------|------------------------------------------------|
| Benjamini-Hochberg (BH)  | False Discovery Rate (FDR) | ❌ No         | ✅ Yes    | General-purpose; balances false positives      |
| Bonferroni Correction     | Family-Wise Error Rate (FWER) | ✅ Yes        | ❌ No     | High-stakes tasks; requires strict precision   |
| Higher Criticism (HC)    | Sparse Signal Detection | ⚠ Depends     | ✅ Yes    | Sparse, noisy data; detecting weak signals     |

---




# 📦 Bonferroni Thresholding with Empirical Null Distribution

This code demonstrates how to dynamically determine the optimal number of documents `k` to retrieve for a RAG system query, using **Bonferroni correction**. We generate an empirical null distribution from random query-document pairs sampled from our own RAG corpus to ensure realistic background noise modeling.

---

## 🔧 Setup & Model Initialization
We start by loading a sentence embedding model and defining our document corpus.

```python
from sentence_transformers import SentenceTransformer
import numpy as np
import random
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

# Load embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Your RAG corpus (use real preprocessed passages)
rag_corpus = [
    "Paris is the capital of France.",
    "Cats are independent animals.",
    "The Eiffel Tower is a famous landmark in Paris.",
    "Quantum mechanics explains the behavior of particles.",
    "Vitamin C helps boost the immune system."
]
```

---

## 🧪 Step 1: Create the Null Distribution
We generate an empirical null distribution by computing similarity scores between random queries and randomly selected unrelated documents from your RAG corpus.

```python
def generate_null_distribution(random_queries, corpus, model, num_pairs=10000):
    print(f"Generating null distribution with {num_pairs} random query-doc pairs...")
    null_similarities = []

    for _ in tqdm(range(num_pairs)):
        q = random.choice(random_queries)
        d = random.choice(corpus)

        q_embed = model.encode(q, convert_to_numpy=True, normalize_embeddings=True)
        d_embed = model.encode(d, convert_to_numpy=True, normalize_embeddings=True)

        sim = np.dot(q_embed, d_embed)  # cosine similarity
        null_similarities.append(sim)

    return np.array(null_similarities)

# Random unrelated queries
random_queries = [
    "How do I reset my password?",
    "What is the weather like in July?",
    "Tips for learning Japanese online",
    "What are the symptoms of flu?",
    "How to cook rice perfectly?"
]

# Generate the null distribution
null_distribution = generate_null_distribution(random_queries, rag_corpus, model, num_pairs=10000)

# Save for reuse
np.save("null_distribution.npy", null_distribution)
```

---

## 🎯 Step 2: Bonferroni Thresholding Function
This function takes in similarity scores and the null distribution to calculate p-values and applies Bonferroni correction.

```python
def bonferroni_thresholding(similarities, null_distribution, alpha=0.05):
    """
    Apply Bonferroni correction to dynamically select top-k documents.
    """
    m = len(similarities)
    p_values = np.array([
        np.mean(null_distribution >= s) for s in similarities
    ])
    threshold = alpha / m
    selected_indices = np.where(p_values <= threshold)[0].tolist()
    return selected_indices, p_values
```

---

## 🔍 Step 3: Query Evaluation and Dynamic `k`
This part simulates how you’d retrieve documents and apply Bonferroni to determine the appropriate number of documents to include.

```python
# Example user query
query = "Tell me about France."
query_embedding = model.encode([query], convert_to_numpy=True, normalize_embeddings=True)

# Compute similarities to all documents in corpus
doc_embeddings = model.encode(rag_corpus, convert_to_numpy=True, normalize_embeddings=True)
similarities = np.dot(doc_embeddings, query_embedding[0])

# Apply Bonferroni thresholding
selected_indices, p_vals = bonferroni_thresholding(similarities, null_distribution, alpha=0.05)

# Display selected results
print(f"\nQuery: {query}\n")
print(f"Selected top-{len(selected_indices)} documents using Bonferroni correction:")
for idx in selected_indices:
    print(f"[Score: {similarities[idx]:.4f} | p = {p_vals[idx]:.4f}] → {rag_corpus[idx]}")
```

---

## 📌 Summary
- This pipeline dynamically selects `k` documents based on statistical significance.
- The **null distribution** reflects background similarity scores.
- **Bonferroni correction** ensures a strict false positive control.

✅ Ideal for high-precision settings like legal, medical, or compliance-related RAG systems.



In [1]:
#todo add about put in cache all the embedding (vector index) and use ANN to find 